# Stereo Camera Calibration Pipeline

This notebook performs complete stereo camera calibration including:
- Chessboard detection with parallel processing
- Robust RANSAC-based intrinsic calibration
- Stereo pair matching and diversity subsampling
- Stereo calibration and rectification
- Comprehensive visualizations

**Features:**
- Result caching for faster re-runs
- Configurable via YAML file
- Modular utilities for reuse in other scripts

## 1. Setup and Configuration

In [ ]:
# Add utils to path
import sys
from pathlib import Path

proj_root = Path.cwd().parent if Path.cwd().name == "calibration" else Path.cwd()
sys.path.insert(0, str(proj_root / "calibration"))

from utils import (
    CalibrationConfig,
    StereoRig,
    build_stereo_pairs_from_detections,
    calibrate_intrinsics_robust,
    calibrate_stereo,
    detect_chessboards_parallel,
    load_calibration,
    plot_imagepoints_heatmap,
    plot_matched_boards,
    plot_rectification_preview,
    plot_stereo_pair_coverage,
    plot_undistortion_comparison,
    save_calibration,
    subsample_stereo_pairs,
)

# Load configuration
config_path = proj_root / "calibration" / "config.yaml"
config = CalibrationConfig.from_yaml(config_path)

print(f"Project root: {proj_root}")
print(f"Configuration loaded from: {config_path}")
print(f"Visualization mode: {config.visualization_mode}")

ImportError: cannot import name 'plot_stereo_pair_coverage' from 'utils' (c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\utils\__init__.py)

## 2. Load Calibration Images

In [2]:
# Construct paths to calibration images
raw_data_folder = proj_root / config.get("data.raw_data_folder")
calib_left_dir = raw_data_folder / config.get("data.calib_left")
calib_right_dir = raw_data_folder / config.get("data.calib_right")

# Collect image paths
images_left = sorted(calib_left_dir.glob("*.png"))
images_right = sorted(calib_right_dir.glob("*.png"))

print(f"Found {len(images_left)} left images in: {calib_left_dir}")
print(f"Found {len(images_right)} right images in: {calib_right_dir}")

assert len(images_left) > 0 and len(images_right) > 0, "No calibration images found!"

Found 19 left images in: c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\data\34759_final_project_raw\calib\image_02\data
Found 19 right images in: c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\data\34759_final_project_raw\calib\image_03\data


## 3. Detect Chessboards

Detect all chessboard patterns in both left and right calibration images using parallel processing.

In [ ]:
pattern_sizes = config.pattern_sizes
max_boards = config.get("detection.max_boards_per_image", 13)
num_workers = config.get("detection.num_workers", -1)

print(f"Detecting chessboards with patterns: {pattern_sizes}")
print(f"Using {num_workers if num_workers > 0 else 'all'} CPU cores\n")

# Detect LEFT
print("=== LEFT CAMERA ===")
corners_left, objpoints_left, indices_left, image_size = detect_chessboards_parallel(
    [str(p) for p in images_left],
    pattern_sizes,
    max_boards=max_boards,
    num_workers=num_workers,
    progress=True,
    cache_dir=cache_dir,
)
print(f"Detected {len(corners_left)} boards in {len(images_left)} images\n")

# Detect RIGHT
print("=== RIGHT CAMERA ===")
corners_right, objpoints_right, indices_right, _ = detect_chessboards_parallel(
    [str(p) for p in images_right],
    pattern_sizes,
    max_boards=max_boards,
    num_workers=num_workers,
    progress=True,
    cache_dir=cache_dir,
)
print(f"Detected {len(corners_right)} boards in {len(images_right)} images")
print(f"\nImage size: {image_size}")

Detecting chessboards with patterns: [(11, 7), (15, 5), (7, 5)]
Using all CPU cores

=== LEFT CAMERA ===


Detecting boards:   0%|          | 0/19 [00:00<?, ?it/s]

Detected 247 boards in 19 images

=== RIGHT CAMERA ===


Detecting boards:   0%|          | 0/19 [00:00<?, ?it/s]

Detected 209 boards in 19 images

Image size: (1392, 512)


## 4. Robust Intrinsic Calibration

Perform robust RANSAC-based intrinsic calibration for each camera individually.

In [4]:
# Initialize stereo rig
rig = StereoRig()

# Populate detections
rig.left.objpoints = objpoints_left
rig.left.imgpoints = corners_left
rig.left.image_indices = indices_left
rig.left.image_size = image_size

rig.right.objpoints = objpoints_right
rig.right.imgpoints = corners_right
rig.right.image_indices = indices_right
rig.right.image_size = image_size

print("Stereo rig initialized with detection data")

Stereo rig initialized with detection data


In [5]:
# Load robust calibration parameters from config
robust_params = config.config.get("robust_calibration", {})

# Setup cache directory for faster re-runs
cache_dir = proj_root / config.get("data.cache_dir")
if cache_dir:
    cache_dir = cache_dir.resolve()
    print(f"Cache directory: {cache_dir}\n")

# Calibrate LEFT camera
print("=" * 60)
print("LEFT CAMERA ROBUST CALIBRATION")
print("=" * 60)

result_left = calibrate_intrinsics_robust(
    rig.left.objpoints,
    rig.left.imgpoints,
    image_size,
    image_indices_list=rig.left.image_indices,
    cache_dir=cache_dir,
    **robust_params,
)

rig.left.K = result_left.K
rig.left.dist = result_left.dist

print("\n" + result_left.summary())

Cache directory: C:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\cache

LEFT CAMERA ROBUST CALIBRATION
[Stage 1] Naive calibration on up to 50 boards (removed stage0=0/247, geom duplicates=0)


Naive RMS:   0%|          | 0/247 [00:00<?, ?it/s]

[Stage 1a] Extrinsic clustering: kept 26 (removed 221)
[Stage 3] RANSAC search
RANSAC: iters=200 subset_size=6 candidates=26


RANSAC:   0%|          | 0/200 [00:00<?, ?it/s]

RANSAC best: inliers=26/26 RMS=0.157
[Stage 4] Post-rejection tightening


Post RMS:   0%|          | 0/26 [00:00<?, ?it/s]

Post-rejection: thr=0.236 removed=2
[Stage 5] Final calibration


Final RMS:   0%|          | 0/24 [00:00<?, ?it/s]


Robust Calibration Summary:
  Total detections         : 247
  Removed stage0           : 0
  Removed duplicates (geom): 0
  Removed duplicates (extr): 221
  Diversity selected       : 26
  Removed pre-rejection    : 0
  Removed post-rejection   : 2
  Final boards             : 24
  RMS naive                : 0.1880 px
  RMS final                : 0.1610 px
  Improvement              : 0.0270 px
  Runtime                  : 63.62 s



In [6]:
# Calibrate RIGHT camera
print("\n" + "=" * 60)
print("RIGHT CAMERA ROBUST CALIBRATION")
print("=" * 60)

result_right = calibrate_intrinsics_robust(
    rig.right.objpoints,
    rig.right.imgpoints,
    image_size,
    image_indices_list=rig.right.image_indices,
    cache_dir=cache_dir,
    **robust_params,
)

rig.right.K = result_right.K
rig.right.dist = result_right.dist

print("\n" + result_right.summary())


RIGHT CAMERA ROBUST CALIBRATION
[Stage 1] Naive calibration on up to 50 boards (removed stage0=0/209, geom duplicates=0)


Naive RMS:   0%|          | 0/209 [00:00<?, ?it/s]

[Stage 1a] Extrinsic clustering: kept 23 (removed 186)
[Stage 3] RANSAC search
RANSAC: iters=200 subset_size=6 candidates=23


RANSAC:   0%|          | 0/200 [00:00<?, ?it/s]

RANSAC best: inliers=22/23 RMS=0.094
[Stage 4] Post-rejection tightening


Post RMS:   0%|          | 0/22 [00:00<?, ?it/s]

Post-rejection: thr=0.138 removed=2
[Stage 5] Final calibration


Final RMS:   0%|          | 0/20 [00:00<?, ?it/s]


Robust Calibration Summary:
  Total detections         : 209
  Removed stage0           : 0
  Removed duplicates (geom): 0
  Removed duplicates (extr): 186
  Diversity selected       : 23
  Removed pre-rejection    : 0
  Removed post-rejection   : 2
  Final boards             : 20
  RMS naive                : 0.1799 px
  RMS final                : 0.0791 px
  Improvement              : 0.1009 px
  Runtime                  : 65.44 s



## 5. Visualize Intrinsic Calibration Results

In [7]:
import random

output_dir = proj_root / config.get("data.output_dir")
output_dir.mkdir(parents=True, exist_ok=True)
vis_mode = config.visualization_mode

# Undistortion comparison - LEFT
sample_left = str(random.choice(images_left))
plot_undistortion_comparison(
    sample_left,
    rig.left.K,
    rig.left.dist,
    camera_name="LEFT",
    output_path=output_dir / "undistortion_left.png",
    mode=vis_mode,
)

# Undistortion comparison - RIGHT
sample_right = str(random.choice(images_right))
plot_undistortion_comparison(
    sample_right,
    rig.right.K,
    rig.right.dist,
    camera_name="RIGHT",
    output_path=output_dir / "undistortion_right.png",
    mode=vis_mode,
)

In [8]:
# Coverage heatmaps
heatmap_bins = config.get("visualization.heatmap_bins", 40)

plot_imagepoints_heatmap(
    result_left.final_imgpoints,
    image_size,
    camera_name="LEFT",
    output_path=output_dir / "heatmap_left.png",
    mode=vis_mode,
    bins=heatmap_bins,
)

plot_imagepoints_heatmap(
    result_right.final_imgpoints,
    image_size,
    camera_name="RIGHT",
    output_path=output_dir / "heatmap_right.png",
    mode=vis_mode,
    bins=heatmap_bins,
)


LEFT Coverage Statistics:
  Total points: 1008
  Total boards: 24
  X range: [163.7, 1373.2] (coverage: 86.9%)
  Y range: [84.4, 465.0] (coverage: 74.4%)

RIGHT Coverage Statistics:
  Total points: 784
  Total boards: 20
  X range: [98.4, 1249.1] (coverage: 82.7%)
  Y range: [82.7, 453.6] (coverage: 72.4%)

RIGHT Coverage Statistics:
  Total points: 784
  Total boards: 20
  X range: [98.4, 1249.1] (coverage: 82.7%)
  Y range: [82.7, 453.6] (coverage: 72.4%)


## 6. Build Stereo Pairs from Detected Boards

Match chessboards between left and right images to create stereo calibration pairs. This uses the already-detected boards from section 3.

In [ ]:
# Build stereo pairs by matching already-detected boards
matching_params = config.config.get("matching", {})
square_size = config.square_size

stereo_objpoints, stereo_imgpoints_left, stereo_imgpoints_right, matching_metadata = build_stereo_pairs_from_detections(
    corners_left,
    objpoints_left,
    indices_left,
    corners_right,
    objpoints_right,
    indices_right,
    dy_thresh=matching_params.get("dy_thresh", 12.0),
    scale_ratio_thresh=matching_params.get("scale_ratio_thresh", 0.25),
    method=matching_params.get("method", "hungarian"),
    cache_dir=cache_dir,
    progress=True,
)

# Scale object points by square size
for objp in stereo_objpoints:
    objp *= float(square_size)

print("\nMatching statistics:")
print(f"  Total pairs processed: {matching_metadata['total_pairs_processed']}")
print(f"  Total boards matched: {matching_metadata['total_boards_matched']}")
print(
    f"  Average matches per pair: {sum(matching_metadata['matches_per_pair']) / len(matching_metadata['matches_per_pair']):.1f}"
)

# Visualize a few example matched pairs
output_dir = proj_root / config.get("data.output_dir")
output_dir.mkdir(parents=True, exist_ok=True)
vis_mode = config.visualization_mode

# TODO: Add visualization of matched pairs using plot_matched_boards
# This would require storing images or image paths in the metadata

Matching chessboards across 19 stereo pairs...

Pair 1/19: 11 matched boards (total so far: 11)
Pair 1/19: 11 matched boards (total so far: 11)
Pair 2/19: 11 matched boards (total so far: 22)
Pair 2/19: 11 matched boards (total so far: 22)
Pair 3/19: 11 matched boards (total so far: 33)
Pair 3/19: 11 matched boards (total so far: 33)
Pair 4/19: 11 matched boards (total so far: 44)
Pair 4/19: 11 matched boards (total so far: 44)
Pair 5/19: 11 matched boards (total so far: 55)
Pair 5/19: 11 matched boards (total so far: 55)
Pair 6/19: 11 matched boards (total so far: 66)
Pair 6/19: 11 matched boards (total so far: 66)
Pair 7/19: 11 matched boards (total so far: 77)
Pair 7/19: 11 matched boards (total so far: 77)
Pair 8/19: 11 matched boards (total so far: 88)
Pair 8/19: 11 matched boards (total so far: 88)
Pair 9/19: 11 matched boards (total so far: 99)
Pair 9/19: 11 matched boards (total so far: 99)
Pair 10/19: 11 matched boards (total so far: 110)
Pair 10/19: 11 matched boards (total s

## 6b. Subsample Stereo Pairs for Diversity

Apply diversity-based subsampling to stereo pairs to reduce redundancy and speed up stereo calibration while maintaining good geometric coverage.

In [32]:
# Subsample stereo pairs for diversity
subsample_config = config.config.get("stereo_subsampling", {})
if subsample_config.get("enabled", True):
    print("\n" + "=" * 60)
    print("STEREO PAIR DIVERSITY SUBSAMPLING")
    print("=" * 60)

    stereo_objpoints_subs, stereo_imgpoints_subs_left, stereo_imgpoints_subs_right, kept_indices = (
        subsample_stereo_pairs(
            stereo_objpoints,
            stereo_imgpoints_left,
            stereo_imgpoints_right,
            rig.left.K,
            rig.left.dist,
            max_pairs=subsample_config.get("max_pairs", 100),
            min_angle_deg=subsample_config.get("min_angle_deg", 10.0),
            min_translation=subsample_config.get("min_translation", 0.05),
            verbose=True,
        )
    )

    print(f"\nKept {len(kept_indices)} pairs from original {total_matched}")

    # Visualize spatial coverage of stereo pairs
    plot_stereo_pair_coverage(
        stereo_imgpoints_left,
        stereo_imgpoints_right,
        image_size,
        kept_indices=kept_indices,
        output_path=output_dir / "stereo_pair_coverage.png",
        mode=vis_mode,
    )
    print("Stereo pair coverage visualization saved")
else:
    stereo_objpoints_subs = stereo_objpoints
    stereo_imgpoints_subs_left = stereo_imgpoints_left
    stereo_imgpoints_subs_right = stereo_imgpoints_right
    print("\nStereo pair subsampling disabled, using all matched pairs")


STEREO PAIR DIVERSITY SUBSAMPLING
Stereo diversity subsampling: 208 → 12 pairs
  Criteria: min_angle=400.0°, min_translation=0.25

Kept 12 pairs from original 208
Stereo diversity subsampling: 208 → 12 pairs
  Criteria: min_angle=400.0°, min_translation=0.25

Kept 12 pairs from original 208


NameError: name 'plot_stereo_pair_coverage' is not defined

## 7. Stereo Calibration and Rectification

In [30]:
# Stereo calibration parameters
stereo_params = config.config.get("stereo", {})

print("\n" + "=" * 60)
print("STEREO CALIBRATION")
print("=" * 60 + "\n")

rig = calibrate_stereo(
    rig,
    stereo_objpoints_subs,
    stereo_imgpoints_subs_left,
    stereo_imgpoints_subs_right,
    image_size,
    **stereo_params,
)

print("\nStereo calibration complete!")
print(f"  RMS error: {rig.stereo_rms:.4f} px")
print(f"  Baseline: {rig.baseline:.4f} m")
print(f"  Left ROI: {rig.left.roi}")
print(f"  Right ROI: {rig.right.roi}")


STEREO CALIBRATION

Stereo calibration with 12 board pairs...
Stereo RMS: 2.1760 px
Baseline: 1.0153 m
Rectification complete. ROIs: L=(1258, 0, 134, 512), R=(0, 0, 0, 0)

Stereo calibration complete!
  RMS error: 2.1760 px
  Baseline: 1.0153 m
  Left ROI: (1258, 0, 134, 512)
  Right ROI: (0, 0, 0, 0)
Stereo RMS: 2.1760 px
Baseline: 1.0153 m
Rectification complete. ROIs: L=(1258, 0, 134, 512), R=(0, 0, 0, 0)

Stereo calibration complete!
  RMS error: 2.1760 px
  Baseline: 1.0153 m
  Left ROI: (1258, 0, 134, 512)
  Right ROI: (0, 0, 0, 0)


## 8. Visualize Rectification Results

In [27]:
# Pick a random stereo pair for visualization
test_idx = random.randint(0, pair_count - 1)
n_guides = config.get("visualization.rectification_guide_lines", 12)

plot_rectification_preview(
    str(images_left[test_idx]),
    str(images_right[test_idx]),
    rig.left.map1,
    rig.left.map2,
    rig.right.map1,
    rig.right.map2,
    roi_left=rig.left.roi,
    roi_right=rig.right.roi,
    n_guides=n_guides,
    output_path=output_dir / f"rectification_preview_{test_idx}.png",
    mode=vis_mode,
)

print(f"Rectification preview saved for image pair {test_idx}")

Rectification preview saved for image pair 1


## 9. Export Calibration Parameters

In [13]:
# Export configuration
export_format = config.get("export.format", "yaml")
output_filename = config.get("export.output_filename", "stereo_calibration.yaml")
output_path = output_dir / output_filename

save_calibration(
    rig,
    output_path,
    fmt=export_format,
    include_metadata=config.get("export.include_metadata", True),
)

print(f"\nCalibration results exported to: {output_path}")
print(f"Format: {export_format}")

Calibration saved to c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\results\stereo_calibration.yaml

Calibration results exported to: c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\results\stereo_calibration.yaml
Format: yaml


## 10. Verify Saved Calibration (Optional)

Load the saved calibration back and verify parameters.

In [14]:
# Load and verify
rig_loaded = load_calibration(output_path)

print("\nVerification:")
print(f"  Left camera calibrated: {rig_loaded.left.is_calibrated()}")
print(f"  Right camera calibrated: {rig_loaded.right.is_calibrated()}")
print(f"  Stereo calibrated: {rig_loaded.is_stereo_calibrated()}")
print(f"  Rectified: {rig_loaded.is_rectified()}")
print(f"  Baseline: {rig_loaded.baseline:.4f} m")

# Compare key parameters
import numpy as np

k_diff = np.linalg.norm(rig.left.K - rig_loaded.left.K)
print(f"  K matrix difference (L2 norm): {k_diff:.2e}")

Calibration loaded from c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\results\stereo_calibration.yaml

Verification:
  Left camera calibrated: True
  Right camera calibrated: True
  Stereo calibrated: True
  Rectified: False
  Baseline: 1.0125 m
  K matrix difference (L2 norm): 0.00e+00


## Summary

Calibration pipeline complete! The following files have been generated:

- Calibration parameters: `{output_dir}/{output_filename}`
- Visualization outputs: `{output_dir}/*.png`

Use `load_calibration()` to load the parameters in other scripts.